# doubledot.ATMS

>  Definition of the ATMS_api class.

In [1]:
%load_ext autoreload
%autoreload 2

In [68]:
#| default_exp ATMS_api

In [69]:
#| exporti
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os
from fastcore.test import test_eq
import glob
import time
import random
from pyisemail import is_email
from dataclasses import dataclass, field
from collections import namedtuple, UserDict

In [70]:
Data_c = namedtuple('Data_c',['contacts','memberships','sales','items'])
data_t = Data_c({}, {}, {}, {})
# data_t.contacts = {'dog':'cat'} # throws error. can't assign to named tuple
data_t.contacts['dog'] = 'cat' # this works though  
data_t


Data_c(contacts={'dog': 'cat'}, memberships={}, sales={}, items={})

In [71]:
#| export


class ATMS_d(dict):
    """A dictionary that allows dot notation access to its keys and values.
    """

    key_l = ['contacts','memberships','sales','items']
    def __init__(self, val=None):
        if val is None:
            val = {'contacts':{},'memberships':{},'sales':{},'items':{}}
        super().__init__(val)

    # dot notation access
    def __getitem__(self, item):
        # print('get with []', item, self)
        if item not in self.key_l:
            raise KeyError(f'\"{item}\" must be one of: {", ".join(self.key_l)}')
        return super().__getitem__(item )
    
    def __setitem__(self, item, value):
        print('set with []')
        if item not in self.key_l:
            raise KeyError(f'\"{item}\" must be one of: {", ".join(self.key_l)}')
        super().__setitem__(item, value)

    # [] notation access
    def __delitem__(self, item):
        super().__delitem__(item)
    
    # [] notation access
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError as k:
            raise AttributeError(k)

    # dot notation access   
    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        try:
            del self[key]
        except KeyError as k:
            raise AttributeError(k)

    def __repr__(self):
        return '<ATMS ' + dict.__repr__(self) + '>'


In [72]:
_at = ATMS_d()
_at['contacts'] = 3
print(_at['contacts'])
# _at['ths'] = 'that'
# _at.ths
# _at.ths = 'then'
_at.keys()

set with []
None


dict_keys(['contacts', 'memberships', 'sales', 'items'])

In [74]:
#| export
## to be used with the ATMS API to hold data dictionaries

# @dataclass
# class ATMS_data:
#     table_l = ['contacts','sales','memberships','items']
#     contacts: dict = field(default_factory=dict)
#     sales: dict = field(default_factory=dict)
#     memberships: dict = field(default_factory=dict)
#     items: dict = field(default_factory=dict)
#     def from_str(self, s:str):
#         if s in ATMS_data.table_l # ['contacts','sales','memberships','items']:
#             return self.__getattribute__(s)
#         else:
#             raise ValueError(f'{s} must be one of contacts, sales, memberships, items')
    

In [75]:
data_c = ATMS_data()
data_c.contacts = {'dog':'cat'}
data_c.from_str('contacts')['woof'] = 'meow'
print(data_c)
data_c.from_str('sales') 

ATMS_data(contacts={'dog': 'cat', 'woof': 'meow'}, sales={}, memberships={}, items={})


{}

In [77]:
#| export
class ATMS_api:
    class_download_dir = os.path.join(os.getcwd(),'atms_download')
    
    @property
    def telus_access_token(self):
        if self._telus_access_token is None:
            self._telus_access_token = ATMS_api.get_atms_authentication()
        return self._telus_access_token

    @staticmethod
    def delete_all_data():
        """ delete all the subdirectories and files in the download directory """
        for dir_path in glob.glob(os.path.join(ATMS_api.class_download_dir, "*")):
            if os.path.isdir(dir_path):
                for file_path in glob.glob(os.path.join(dir_path, "*")):
                    if os.path.isfile(file_path):
                        os.remove(file_path)
                        print(f"File '{file_path}' deleted successfully.")
                os.removedirs(dir_path) 
                print(f"Directory '{dir_path}' deleted successfully.")  
        print(f"Directory '{ATMS_api.class_download_dir}' deleted successfully.")   



    @staticmethod    
    def get_atms_authentication():
        """get access token for ATMS API

        Returns:
            response object: response object from the API call 
        """
        vantix_url = "http://crm-api-telus.atmsplus.com/auth"
        
        with open('secrets.json') as f:
            secrets = json.load(f)

        payload = json.dumps({
            "username": secrets['vantix_user'],
            "password": secrets['vantix_pw'],
            "rememberMe": True
        })
        headers = {
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", vantix_url, headers=headers, data=payload)
        assert response.status_code == 200, f"response code is {response.status_code}, not 200"
        return response.json().get('access_token')

                
    #make a function that takes a string and returns a string with the original semi-colon separated emails replaced with a list of emails with quotes around each

    @staticmethod
    def _mutate_email_list(s:str # a string like: ' ... "emails": [ ... "address": "pam.jenkins@blackgold.ca;don.what@this.com"}], ...  '
                            ) -> str : # the same string but with ' ... "address": ["pam.jenkins@blackgold.ca", "don.what@this.com"]} ...
        """ 

        """
        pat_s = r"""\"emails\": \[.*?address\": (\"(?P<email1>.*?)\")"""
        pattern=re.compile(pat_s)
        matches = re.search(pattern,s) 
        if matches and matches.group(0) and matches.group(1) and matches.group(2):
            og_emails_list_s = matches.group(2)
            # emails_l = [f"\"{email}\"" for email in og_emails_list_s.split(';')]
            emails_l = [f"\"{email}\"" for email in og_emails_list_s.split(';') if is_email(email)]
            # emails_l = [f"{email}" for email in og_emails_list_s.split(';')]
            emails_list_s = '[' +','.join(emails_l)+']'
            return re.sub(matches.group(1), emails_list_s,s)
        else:
            return s



## `__init__`

In [78]:
#| export
@patch
def __init__(self:ATMS_api, data_dir: str = None ):
    # if telus_access_token is None:
    #     self.telus_access_token = ATMS_api.get_atms_authentication()
    # else:
    #     self.telus_access_token = telus_access_token
    self._telus_access_token = None


    # object keys shoud be ['memberships', 'sales', 'items', 'contacts']
    # i'eg ; https://goodcode.io/articles/python-dict-object/
    # self.data = ATMS_data()
    # self.obj_d = {'contacts': self.data.contacts, 'sales': self.data.sales, 'memberships': self.data.memberships, 'items': self.data.items}
    self.obj_d = ATMS_d()

    # create unique download directory per instance
    if not os.path.exists(ATMS_api.class_download_dir):
        os.makedirs(ATMS_api.class_download_dir)
        print(f"Directory 'atms_download' created successfully.")
    else:
        print(f"Directory 'atms_download' already exists.")

    if data_dir is None:
        # generate a unique directory name with a random string
        random_string = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=8))
        self.download_dir  = os.path.join(ATMS_api.class_download_dir, random_string)

        # Check if the new director already exists in the directory
        while os.path.exists(self.download_dir):
            # If the directory name already exists, generate a new one
            random_string  = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=8))
            self.download_dir  = os.path.join(ATMS_api.class_download_dir, random_string)
        
        os.mkdir(self.download_dir)
        self.id = random_string
    else:
        self.download_dir  = os.path.join(ATMS_api.class_download_dir, data_dir)
        self.id = data_dir
        
    print("my id is", self.id)


## write_data_to_json_files 

In [114]:
#| export

@patch
def write_data_to_json_files(self:ATMS_api):
    """ set the atms_id and download_dir for the instance """
    # self.download_dir  = os.path.join(ATMS_api.class_download_dir, atms_id)
    for obj_key, obj_v in self.obj_d.items():
        first = True
        with open(os.path.join(self.download_dir, f"{obj_key}.json"), 'w') as f:
            for l in obj_v:
                if first: 
                     f.write('[\n')
                     f.write(json.dumps(l))
                     
                     first = False
                else:
                      f.write(', \n' + json.dumps(l) )
            f.write(']\n')
            print(f"File '{obj_key}.json' written successfully.")



In [81]:
atms = ATMS_api()

Directory 'atms_download' already exists.
my id is 2pj08794


In [82]:
#| export
@patch
def set_atms_id(self:ATMS_api, atms_id:str):
    """ set the atms_id and download_dir for the instance """
    self.atms_id = atms_id
    self.download_dir  = os.path.join(ATMS_api.class_download_dir, atms_id)


## list_files

In [83]:
#| export
@patch    
def list_files(self:ATMS_api):
    """ returns list of files in download folder """
    print('id is:', self.id)
    if os.path.exists(self.download_dir):
        return os.listdir(self.download_dir)
    else:
        return []


## clean_data_dir

In [84]:
#|export    
@patch
def clean_data_dir(self:ATMS_api,
                    obj_s: str = None):
    glob_s = os.path.join(self.download_dir, f"*{obj_s if obj_s else ''}*.json")
    file_l = glob.glob(glob_s)
    # print(file_l)
    for file_path in file_l:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"File '{file_path}' deleted successfully.")


## get_telus_data

In [85]:
#|export    
@patch
def get_telus_data(self:ATMS_api, 
                    obj: str, # telus endpoint 
                    offset :int = 0, # the row to begin retrieval
                    since_date: str = "", # optionally, the date from which to retrieve
                    count :int =1, # the number of rows to retrieve
                    contact_id: int = None # the contact id to retrieve
                    ):
    """retrieve data from ATMS API, should be private method

    Args:
        obj (string): api endpoint to retrieve data from
        offset (int, optional): first row to begin retrieval. Defaults to 0.
        count (int, optional): number of rows to retrieve. Defaults to 1000.

    Returns:
        a dict with two keys: "response" and "done". 
        "response" is the response object from the API call. "done" is a boolean indicating whether there are more records to retrieve.
    """
    vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}?offset={offset}&count={count}"
    if len(since_date)> 0:
        print("ATMS_api.get_telus_data: since_date is", since_date)
        vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}/lastupdate?count={count}&offset={offset}&updateDate={since_date}"
    
    if contact_id:
        print('we have contact_id', contact_id, 'and obj is', obj)
        ## if contact id and obj == 'contacts' everything else is ignored
        if obj=='contacts':
            print('we have contact_id and obj is contacts')
            vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}/{contact_id}"
            print("ATMS_api.get_telus_data : ", vantix_data_url)

        ## if contact id everything else is ignored
        elif obj=='sales' or obj=='memberships':
            vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}/contact/{contact_id}"
        else:
            raise ValueError("contact_id is only valid for 'contacts' and 'sales'")

    v_headers = {'Authorization': f"Bearer {self.telus_access_token}"}

    print(vantix_data_url)    
    # response = requests.request("GET", vantix_data_url, headers=v_headers, data={}).json()
    response = requests.request("GET", vantix_data_url, headers=v_headers, data={})
    
    # inform caller we're done if we get fewer records than requested
    # BUT there could still be an error in the response
    return {"response": response, "done":  response.status_code!=200 or len(response.json()) < count}



## retrieve_and_clean

In [86]:
#|export    
@patch
def retrieve_and_clean(self:ATMS_api, 
                        obj : str = 'contacts', # ATMS object to retrieve
                        initial_offset : int =0, # start retrieval at row initial_offset
                        rows_per_batch :int =1000, # number records retrieved at once
                        since_date : str = "", # if given, it will be used instead of `initial_offset` 
                        max_rows :int = 0 # maximum number of rows to retrieve
                        ):
    """Retrieve data from ATMS API, clean data and write to file"""
    self.write_obj_to_file(obj, initial_offset, rows_per_batch, since_date, max_rows)
    self.clean_data_file(obj)
    self.load_data_file_to_dict(obj)



## `fetch_data_by_contactIds`

In [87]:
#| export
@patch
def fetch_data_by_contactIds(
        self: ATMS_api,
        obj_s : str = 'contacts', # string to describe object to fetch
        id_l : list = [], # list of contactIds
        store_data_b : bool = True # if True, store data in self.obj_d
    ):
    """ retrieve data from ATMS by contactId for a list of contactIds for the given object.
        returns a list of dicts, one for each contactId
    """
    l = []
    # list unique
    id_s = set(id_l)
    for contact_id in id_s: #pelton_df.contactKey.unique():
        print(contact_id)
        r = self.get_telus_data(obj_s, contact_id=contact_id)
        data_d = r['response'].json()
        print(data_d)
        l.append(data_d)
        sleep(1)
    if store_data_b:
        self.obj_d[obj_s] = l
    return l ### write to obj_d ??

## ``write_obj_d_to_file``

In [88]:
def write_obj_d_to_file(
        self: ATMS_api,
):
    """ write the obj_d to file """
    for obj_s in self.obj_d:
        self.write_obj_to_file(obj_s)

## write_obj_to_file


In [89]:
#|export    
@patch
def write_obj_to_file(self:ATMS_api, 
                        obj : str = 'contacts', # ATMS object to retrieve
                        initial_offset : int =0, # start retrieval at row initial_offset
                        rows_per_batch :int =1000, # number records retrieved at once
                        since_date : str = "", # if given, it will be used instead of `initial_offset` 
                        max_rows :int = 0 # maximum number of rows to retrieve
                        ):
    """Retrieve data from ATMS API and write to file
        private method

    Args:
        obj (string): a valid ATMS REST API object
        rows_per_batch (int, optional): maximum number of rows to retieve. Defaults to 1000.
    """
    done = False
    # offset is the starting row for the next batch
    offset = initial_offset 

    filename_s = f'atms_{obj}.json'
    print('download dir is: ', self.download_dir)
    file_path_s = os.path.join(self.download_dir, filename_s)
    # print("Writing to file: ", file_path_s)
    
    # if max_rows is 0, we'll retrieve all rows
    with open(file_path_s, 'w') as f:
        f.write("[ \n")
        num_rows_for_next_batch = min(rows_per_batch, max_rows ) if max_rows > 0 else rows_per_batch

        #max_remaining_rows will be decremented as we retrieve batches
        max_remaining_rows = max_rows if max_rows > 0 else rows_per_batch

        #num_rows_for_next_batch will be 
        num_rows_for_next_batch = min(rows_per_batch, max_remaining_rows) 
        first_line = True
        while (not done and (num_rows_for_next_batch > 0)):

            # read another batch
            
            print(f"resp_d = self.get_telus_data({obj},offset={offset}, count= {num_rows_for_next_batch}, since_date={since_date})")
            resp_d = self.get_telus_data(obj,offset=offset, count= num_rows_for_next_batch, since_date=since_date)
            print("resp_d.keys(): ", resp_d.keys())
            if resp_d['response'].status_code != 200:
                print(f"Error retrieving data: {resp_d['response'].status_code}")
                print(f"Error retrieving data: {resp_d['response'].json()}")
                raise ValueError(f"Error retrieving data: {resp_d['response'].status_code}")
            obj_l = resp_d['response'].json()
            done = resp_d['done'] ###### if done is set to True, we'll exit the loop
            print(f"done: {done}, resp_d.json() : {resp_d['response']}")
            
            # assert len(obj_l)>0, f"response: {resp_d}"
            try:
                s = json.dumps(obj_l[0])
            except:
                print("Error converting to json: resp_d ", resp_d.json())
                print("Error converting to json: obj_l ", obj_l)
                raise ValueError(f"Error converting to json {obj_l}")
            for o in obj_l[1:]:
                try:
                    s += ',\n' +json.dumps(o) 
                except:
                    print("Error converting to json: ", obj_l)
                    raise ValueError(f"Error converting to json {o}")
            # s = ",\n".join([json.dumps(o) for o in obj_l])
            
            if first_line:
                f.write(s)
                first_line = False
            else:
                f.write(",\n"+s)
            offset += rows_per_batch
            max_remaining_rows = max_rows - (offset - initial_offset) if max_rows > 0 else rows_per_batch
            num_rows_for_next_batch = min(rows_per_batch, max_remaining_rows)
        f.write("\n]")




## clean_data_file 

In [90]:
#| export
@patch
def clean_data_file(self:ATMS_api, 
                    obj_s : str
                    ): 
    """ Massage formatting to work with Salesfore Bulk API
    
    """
    # read original contacts file   
    in_filename_s = f'atms_{obj_s}.json'
    in_file_path_s = os.path.join(self.download_dir, in_filename_s)
    print("cleaning_data_file - download dir is: ", self.download_dir)
    # print("Cleaning file: ", in_file_path_s)
    try:
        with open(in_file_path_s,'r') as f:
            # write modified contacts file 
            out_filename_s = f'atms_transformed_{obj_s}.json'
            out_file_path_s = os.path.join(self.download_dir, out_filename_s)
            print("creating file: ", out_file_path_s)
            with open(out_file_path_s,'w') as f2:
                s = f.read()
                for l in s.split('\n'):
                    # remove "O`Brien" problem
                    l2 = re.sub('\u2019',"'",l)
                    # fix emails
                    new_s = ATMS_api._mutate_email_list(l2)+'\n'
                    f2.write(new_s)
            print(f"Finished cleaning {in_filename_s} -> {out_file_path_s}")
    except FileNotFoundError:
        print('the files in our download dir:', os.listdir(self.download_dir) )
        print('our in_file_path: ', in_file_path_s)
            



## load_data_to_dict

In [91]:
#| export
@patch
def load_data_file_to_dict(
        self:ATMS_api,
        obj_s : str # ATMS object. eg. contacts|items|memberships|membership
        ):
        """ `load_data_file_to_dict` will attempt to load a cleaned json file into a dict for future parsing. 
        If the cleaned file doesn't exist, it will look for a dirty one to clean.
        If the dirty once doesn't exist, it will raise an exception. It won't be downloaded because we don't know how much to get.
        
        """
        file_name_s = f'atms_transformed_{obj_s}.json'
        file_path_s = os.path.join(self.download_dir, file_name_s)
        print('ATMS_api - Attempting to load: ', file_path_s, ' into dict')
        data=""
        try:
            with open(file_path_s, 'r') as f:
                data = f.read()
        except FileNotFoundError:  # clean file not found
            print("File not found. Check that the dirty file is there")
            dirty_file_name_s = f'atms_{obj_s}.json'
            dirty_file_path_s = os.path.join(self.download_dir, dirty_file_name_s)

            if os.path.exists(dirty_file_path_s):
                print(f"found dirty file: {dirty_file_path_s}")
                self.clean_data_file(obj_s)
                with open(file_path_s,'r') as f:
                    data = f.read()
            else:
                raise FileNotFoundError # we give up
        finally: # this will be executed regardless of first 'try' failing or not
            if len(data) > 0:
                try: # data might be buggy
                    self.obj_d[obj_s] = json.loads(data)
                    print(f"ATMS_api: loaded {len(self.obj_d[obj_s])} {obj_s} into dict")
                except json.JSONDecodeError:
                    print("We've got buggy data, or something")
                    raise Exception('Data is not JSON formatted')
            # assert obj_s in self.obj_d, f" '{obj_s}' not in {self.obj_d.keys()}"

        

## Get Pick list for MembershipTerm.memberType

In [92]:
@patch 
def get_pick_list_for_MembeshipTerms_membershipType( self: ATMS_api):
    """ utility func to get picklist values for 'membershipTyp'
    this needs a full download to work"""
    self.retrieve_and_clean('memberships')
    print('\n'.join(set(jp.search("[].membershipTerms[].membershipType", self.obj_d['memberships']))))

## Test Stuff

In [93]:
# atms = ATMS_api()

In [94]:
# pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167 ]
# atms.fetch_data_by_contactIds('contacts', pelton_ids) 

In [95]:
# atms.retrieve_and_clean('contacts', max_rows=1000, since_date='2020-01-01')
# atms.retrieve_and_clean('memberships', max_rows=1000, since_date='2020-01-01')
# atms.retrieve_and_clean('items', max_rows=1000, since_date='2020-01-01')

In [96]:
#| export

"playtime"

'playtime'

In [97]:
show_doc(ATMS_api.delete_all_data)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L52){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.delete_all_data

>      ATMS_api.delete_all_data ()

delete all the subdirectories and files in the download directory

In [98]:
# ATMS_api.delete_all_data()

In [99]:
show_doc(ATMS_api.get_telus_data)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L191){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.get_telus_data

>      ATMS_api.get_telus_data (obj:str, offset:int=0, since_date:str='',
>                               count:int=1, contact_id:int=None)

retrieve data from ATMS API, should be private method

Args:
    obj (string): api endpoint to retrieve data from
    offset (int, optional): first row to begin retrieval. Defaults to 0.
    count (int, optional): number of rows to retrieve. Defaults to 1000.

Returns:
    a dict with two keys: "response" and "done". 
    "response" is the response object from the API call. "done" is a boolean indicating whether there are more records to retrieve.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| obj | str |  | telus endpoint |
| offset | int | 0 | the row to begin retrieval |
| since_date | str |  | optionally, the date from which to retrieve |
| count | int | 1 | the number of rows to retrieve |
| contact_id | int | None | the contact id to retrieve |

In [100]:
show_doc(ATMS_api.write_obj_to_file)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L283){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.write_obj_to_file

>      ATMS_api.write_obj_to_file (obj:str='contacts', initial_offset:int=0,
>                                  rows_per_batch:int=1000, since_date:str='',
>                                  max_rows:int=0)

Retrieve data from ATMS API and write to file
    private method

Args:
    obj (string): a valid ATMS REST API object
    rows_per_batch (int, optional): maximum number of rows to retieve. Defaults to 1000.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| obj | str | contacts | ATMS object to retrieve |
| initial_offset | int | 0 | start retrieval at row initial_offset |
| rows_per_batch | int | 1000 | number records retrieved at once |
| since_date | str |  | if given, it will be used instead of `initial_offset` |
| max_rows | int | 0 | maximum number of rows to retrieve |

In [101]:
show_doc(ATMS_api.list_files)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L167){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.list_files

>      ATMS_api.list_files ()

returns list of files in download folder

In [102]:
show_doc(ATMS_api.load_data_file_to_dict)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L397){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.load_data_file_to_dict

>      ATMS_api.load_data_file_to_dict (obj_s:str)

`load_data_file_to_dict` will attempt to load a cleaned json file into a dict for future parsing. 
If the cleaned file doesn't exist, it will look for a dirty one to clean.
If the dirty once doesn't exist, it will raise an exception. It won't be downloaded because we don't know how much to get.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| obj_s | str | ATMS object. eg. contacts\|items\|memberships\|membership |

In [103]:
show_doc(ATMS_api.clean_data_file)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L362){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.clean_data_file

>      ATMS_api.clean_data_file (obj_s:str)

Massage formatting to work with Salesfore Bulk API

In [104]:
show_doc(ATMS_api.clean_data_dir)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/ATMS_api.py#L178){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.clean_data_dir

>      ATMS_api.clean_data_dir (obj_s:str=None)

In [105]:
#| hide
import nbdev; nbdev.nbdev_export()

SyntaxError: expected ':' (<unknown>, line 11)